输入输出通常可以划分为几个大类：读取文本文件和其他更高效的磁盘存储格式，加载数据库中的数据，利用Web API操作网络资源。

In [226]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

# 6.1 读写文本格式的数据

pandas提供了一些用于将表格型数据读取为DataFrame对象的函数，其中read_csv和read_table可能会是今后用得最多的。

因为工作中实际碰到的数据可能十分混乱，一些数据加载函数（尤其是read_csv）的选项逐渐变得复杂起来。面对不同的参数，感到头痛很正常（read_csv有超过50个参数）。pandas文档有这些参数的例子，如果你感到阅读某个文件很难，可以通过相似的足够多的例子找到正确的参数。

其中一些函数，比如pandas.read_csv，有类型推断功能，因为列数据的类型不属于数据类型。也就是说，不需要指定列的类型到底是数值、整数、布尔值，还是字符串。其它的数据格式，如HDF5、Feather和msgpack，会在格式中存储数据类型。

日期和其他自定义类型的处理需要多花点工夫才行。首先来看一个以逗号分隔的（CSV）文本文件 `examples/ex1.csv`

由于该文件以逗号分隔，所以可以使用read_csv将其读入一个DataFrame：

In [227]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


还可以使用read_table，并指定分隔符：

In [228]:
df = pd.read_table('examples/ex1.csv', sep=',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


并不是所有文件都有标题行。比如 `examples/ex2.csv` 这个文件：

```
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
```

读入该文件的办法有两个。可以让pandas为其分配默认的列名，也可以自己定义列名:

In [229]:
pd.read_csv('examples/ex2.csv', header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [230]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


假设希望将message列做成DataFrame的索引。可以明确表示要将该列放到索引4的位置上，也可以通过index_col参数指定"message"：

In [231]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可，以文件 `examples/csv_mindex.csv` 为例：

In [232]:
pd.read_csv('examples/csv_mindex.csv', index_col=['key1','key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

有些情况下，有些表格可能不是用固定的分隔符去分隔字段的（空白符或其它模式）。比如 `examples/ex3.csv` 这个文本文件：

In [233]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

虽然可以手动对数据进行规整，这里的字段是被数量不同的空白字符间隔开的。这种情况下，可以传递一个正则表达式作为read_table的分隔符。可以用正则表达式表达为 `\s+` ，于是有：

In [234]:
pd.read_table('examples/ex3.txt', sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


这里，由于列名比数据行的数量少，所以read_table推断第一列应该是DataFrame的索引。

这些解析器函数还有许多参数可以帮助处理各种各样的异形文件格式。比如说，可以用skiprows跳过文件 `examples/ex4.csv` 的第一行、第三行和第四行：

In [235]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有（空字符串），要么用某个标记值表示。默认情况下，pandas会用一组经常出现的标记值进行识别，比如NA及NULL：

In [236]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [237]:
result.isnull()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


na_values可以用一个列表或集合的字符串表示缺失值：

In [238]:
result = pd.read_csv('examples/ex5.csv', na_values=['filled_value'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


字典的各列可以使用不同的NA标记值：

In [239]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件

在处理很大的文件时，或找出大文件中的参数集以便于后续处理时，可能只想读取文件的一小部分或逐块对文件进行迭代。

在看大文件之前，先设置pandas显示地更紧些：

In [240]:
pd.options.display.max_rows = 10

然后有：

In [241]:
pd.read_csv('examples/ex6.csv')

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


如果只想读取几行（避免读取整个文件），通过nrows进行指定即可：

In [242]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


要逐块读取文件，可以指定chunksize（行数）：

In [243]:
# 返回一个可迭代对象
pd.read_csv('examples/ex6.csv', chunksize=1000)

read_csv所返回的这个TextParser对象可以根据chunksize对文件进行逐块迭代。比如说可以迭代处理ex6.csv，将值计数聚合到"key"列中，如下所示：

In [244]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

TextParser还有一个get_chunk方法，可以读取任意大小的块。

## 将数据写出到文本格式

数据也可以被输出为分隔符格式的文本。先看看之前读过的一个CSV文件：

In [245]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


利用DataFrame的to_csv方法，可以将数据写到一个以逗号分隔的文件中：

In [246]:
data.to_csv('examples/out.csv')

pd.read_csv('examples/out.csv', index_col=[0])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


当然，还可以使用其他分隔符（由于这里直接写出到sys.stdout，所以仅仅是打印出文本结果而已）：

In [247]:
import os
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


缺失值在输出结果中会被表示为空字符串。有时可能希望将其表示为别的标记值：

In [248]:
data.to_csv(sys.stdout, sep='|', na_rep='Null')

|something|a|b|c|d|message
0|one|1|2|3.0|4|Null
1|two|5|6|Null|8|world
2|three|9|10|11.0|12|foo


如果没有设置其他选项，默认情况下则会写出行和列的标签。当然，它们也都可以被禁用：

In [249]:
data.to_csv(sys.stdout, na_rep='Null', index=False, header=False)

one,1,2,3.0,4,Null
two,5,6,Null,8,world
three,9,10,11.0,12,foo


此外，你还可以只写出一部分的列，并以指定的顺序排列：

In [250]:
data.to_csv(sys.stdout, index=False, na_rep='Null', columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,Null
9,10,11.0


Series也有一个to_csv方法：

In [251]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
# ts, type(dates)
ts.to_csv('examples/tseries.csv')

pd.read_csv('examples/tseries.csv')

,Unnamed: 0,0
0,2000-01-01,0
1,2000-01-02,1
2,2000-01-03,2
3,2000-01-04,3
4,2000-01-05,4
5,2000-01-06,5
6,2000-01-07,6


## 处理分隔符格式

大部分存储在磁盘上的表格型数据都能用pandas.read_table进行加载。然而，有时还是需要做一些手工处理。由于接收到含有畸形行的文件而使read_table出毛病的情况并不少见。为了说明这些基本工具，看看下面这个 `examples/ex7.csv` 文件：

"a","b","c"

"1","2","3"

"1","2","3"

对于任何单字符分隔符文件，可以直接使用Python内置的csv模块。将任意已打开的文件或文件型的对象传给csv.reader：

In [252]:
import csv

f  = open('examples/ex7.csv')
reader = csv.reader(f)

for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


现在，为了使数据格式合乎要求，需要对其做一些整理工作。

首先，读取文件到一个多行的列表中：

In [253]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

然后，将这些行分为标题行和数据行：

In [254]:
header, values = lines[0], lines[1:]

然后，可以用字典构造式和zip(*values)，后者将行转置为列，创建数据列的字典：

In [255]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV文件的形式有很多。只需定义csv.Dialect的一个子类即可定义出新格式（如专门的分隔符、字符串引用约定、行结束符等）：

In [256]:
f  = open('examples/ex7.csv')

class MyDialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
reader = csv.reader(f, dialect=MyDialect)

for i in reader:
    print(i)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


各个CSV语支的参数也可以用关键字的形式提供给csv.reader，而无需定义子类：

In [257]:
# 读取不到数据，因为文件中不存在分隔符 |
reader = csv.reader(f, delimiter='|')

reader.line_num

0

> 笔记：对于那些使用复杂分隔符或多字符分隔符的文件，csv模块就无能为力了。这种情况下，你就只能使用字符串的split方法或正则表达式方法re.split进行行拆分和其他整理工作了。

要手工输出分隔符文件，可以使用csv.writer。它接受一个已打开且可写的文件对象以及跟csv.reader相同的那些语支和格式化选项：

In [258]:
with open('examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=MyDialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))
pd.read_csv('examples/mydata.csv', dialect=MyDialect)

,one,two,three
0,1,2,3
1,4,5,6
2,7,8,9


## JSON数据

JSON（JavaScript Object Notation的简称）已经成为通过HTTP请求在Web浏览器和其他应用程序之间发送数据的标准格式之一。它是一种比表格型文本格式（如CSV）灵活得多的数据格式。下面是一个例子：

In [259]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [260]:
除其空值null和一些其他的细微差别（如列表末尾不允许存在多余的逗号）之外，JSON非常接近于有效的Python代码。基本类型有对象（字典）、数组（列表）、字符串、数值、布尔值以及null。对象中所有的键都必须是字符串。许多Python库都可以读写JSON数据。下面将使用json，因为它是构建于Python标准库中的。通过json.loads即可将JSON字符串转换成Python形式：

SyntaxError: invalid character in identifier (<ipython-input-260-0d0c9d6e7b0a>, line 1)

In [208]:
import json

result = json.loads(obj)

result  # json数据在Python中的表现形式为字典

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

json.dumps则将Python对象转换成JSON格式：

In [210]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

如何将（一个或一组）JSON对象转换为DataFrame或其他便于分析的数据结构就由你决定了。最简单方便的方式是：向DataFrame构造器传入一个字典的列表（就是原先的JSON对象），并选取数据字段的子集：

In [212]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])
siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


pandas.read_json可以自动将特别格式的JSON数据集转换为Series或DataFrame。例如：

In [215]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


第7章中关于USDA Food Database的那个例子进一步讲解了JSON数据的读取和处理（包括嵌套记录）。

如果需要将数据从pandas输出到JSON，可以使用to_json方法：

In [219]:
print(data.to_json())

print()

print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML和HTML：Web信息收集

Python有许多可以读写常见的HTML和XML格式数据的库，包括lxml、Beautiful Soup和html5lib。lxml的速度比较快，但其它的库处理有误的HTML或XML文件更好。

pandas有一个内置的功能，read_html，它可以使用lxml和Beautiful Soup自动将HTML文件中的表格解析为DataFrame对象。为了进行展示，我从美国联邦存款保险公司下载了一个HTML文件（pandas文档中也使用过），它记录了银行倒闭的情况。首先，你需要安装read_html用到的库：

```
conda install lxml

pip install beautifulsoup4 html5lib
```

如果你用的不是conda，可以使用``pip install lxml``。

pandas.read_html有一些选项，默认条件下，它会搜索、尝试解析<table\>标签内的表格数据。结果是一个列表的DataFrame对象：

In [225]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

failures = tables[0]

failures.head()


ImportError: lxml not found, please install it